# TP

In [124]:
import pandas as pd
import numpy as np

In [126]:
df_objets = pd.read_csv('objets-trouves-restitution.csv', sep=';')

In [127]:
df_objets.head()

,Date,Date et heure de restitution,Gare,Code UIC,Nature d'objets,Type d'objets,Type d'enregistrement
0,2018-08-16T08:46:55+02:00,2018-08-16T10:52:08+02:00,Nantes,87481002.0,Téléphone portable,"Appareils électroniques, informatiques, appare...",Objet trouvé
1,2018-08-16T08:50:34+02:00,NaN,Redon,87471300.0,"Casque (vélo, moto)","Vélos, trottinettes, accessoires 2 roues",Objet trouvé
2,2018-08-16T09:11:18+02:00,NaN,Béziers,87781005.0,"Sac d'enseigne (plastique, papier, …)","Bagagerie: sacs, valises, cartables",Objet trouvé
3,2018-08-16T09:19:47+02:00,NaN,Rennes,87471003.0,"Manteau, veste, blazer, parka, blouson, cape","Vêtements, chaussures",Objet trouvé
4,2018-08-16T09:23:18+02:00,NaN,Paris Montparnasse,87391003.0,Sac à dos,"Bagagerie: sacs, valises, cartables",Objet trouvé


In [130]:
doublons = df_objets.duplicated().sum()

# Affiche le nombre de doublons
print(f"Nombre de doublons : {doublons}")

f_objets = df_objets.drop_duplicates()

Nombre de doublons : 80


In [132]:
print(df_objets.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907276 entries, 0 to 907275
Data columns (total 7 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Date                          907276 non-null  object 
 1   Date et heure de restitution  350898 non-null  object 
 2   Gare                          906197 non-null  object 
 3   Code UIC                      906197 non-null  float64
 4   Nature d'objets               907276 non-null  object 
 5   Type d'objets                 907276 non-null  object 
 6   Type d'enregistrement         907276 non-null  object 
dtypes: float64(1), object(6)
memory usage: 48.5+ MB
None


Il y a **90725 observations**, **7 colonnes** dont seul **la colonne Code UIC a pour type un float, les autres étant du texte**.

On remarque également que pour toutes les colonnes, le nombre non null de valeur n'est pas toujours égal au nombre d'observations. Donc **il manque des valeurs dans les colonnes Code UIC, Gare, et Date et jeure de restitution**

In [134]:
print(df_objets.describe(include=['object']))

                             Date Date et heure de restitution  \
count                      907276                       350898   
unique                     904783                       350213   
top     2017-07-04T07:29:58+02:00    2023-09-27T02:00:22+02:00   
freq                            4                           42   

                      Gare              Nature d'objets  \
count               906197                       907276   
unique                 181                          132   
top     Paris Gare de Lyon  Porte-monnaie, portefeuille   
freq                 70984                        82184   

                              Type d'objets Type d'enregistrement  
count                                907276                907276  
unique                                   16                     1  
top     Bagagerie: sacs, valises, cartables          Objet trouvé  
freq                                 289118                907276  


In [136]:
# Modifier "Type d'enregistrement" en "Objet restitué" si "Date et heure de restitution" n'est pas NaN
df_objets['Type d\'enregistrement'] = np.where(
    df_objets['Date et heure de restitution'].notna(),
    'Objet restitué',
    df_objets['Type d\'enregistrement'])


In [138]:
# Vérifie si chaque Gare a un unique Code UIC
gare_unique_uic = df_objets.groupby("Gare")["Code UIC"].nunique()
gare_incorrecte = gare_unique_uic[gare_unique_uic > 1]

# Vérifie si chaque Code UIC correspond à une unique Gare
uic_unique_gare = df_objets.groupby("Code UIC")["Gare"].nunique()
uic_incorrect = uic_unique_gare[uic_unique_gare > 1]

# Affiche les résultats
if not gare_incorrecte.empty:
    print("Incohérences trouvées pour les Gares avec plusieurs Codes UIC:")
    print(gare_incorrecte)
else:
    print("Chaque Gare a un unique Code UIC.")

if not uic_incorrect.empty:
    print("Incohérences trouvées pour les Codes UIC avec plusieurs Gares:")
    print(uic_incorrect)
else:
    print("Chaque Code UIC correspond à une unique Gare.")


Chaque Gare a un unique Code UIC.
Chaque Code UIC correspond à une unique Gare.


In [142]:
df_objets = df_objets.drop(columns=["Code UIC"])
df_objets.head(1)

KeyError: "['Code UIC'] not found in axis"

In [144]:
# Extraire l'année, le mois, le jour et le décalage horaire en utilisant des sous-chaînes de 'Date'
df_objets['Année'] = df_objets['Date'].str[:4]         # Les 4 premiers caractères pour l'année
df_objets['Mois'] = df_objets['Date'].str[5:7]         # Les caractères 5 à 6 pour le mois
df_objets['Jour'] = df_objets['Date'].str[8:10]        # Les caractères 8 à 9 pour le jour
df_objets['Heure'] = df_objets['Date'].str[11:19]         # Les caractères 11 à 18 pour l'heure:minute:seconde
df_objets['Décalage_Horaire'] = df_objets['Date'].str[-6:]  # Les 6 derniers caractères pour le décalage horaire

# Affichage des premières lignes pour vérifier les nouvelles colonnes
print(df_objets.head())

                        Date Date et heure de restitution                Gare  \
0  2018-08-16T08:46:55+02:00    2018-08-16T10:52:08+02:00              Nantes   
1  2018-08-16T08:50:34+02:00                          NaN               Redon   
2  2018-08-16T09:11:18+02:00                          NaN             Béziers   
3  2018-08-16T09:19:47+02:00                          NaN              Rennes   
4  2018-08-16T09:23:18+02:00                          NaN  Paris Montparnasse   

                                Nature d'objets  \
0                            Téléphone portable   
1                           Casque (vélo, moto)   
2         Sac d'enseigne (plastique, papier, …)   
3  Manteau, veste, blazer, parka, blouson, cape   
4                                     Sac à dos   

                                       Type d'objets Type d'enregistrement  \
0  Appareils électroniques, informatiques, appare...        Objet restitué   
1           Vélos, trottinettes, accessoires 2 rou

In [146]:
print(df_objets['Décalage_Horaire'].unique())

['+02:00' '+01:00']


In [148]:
print(df_objets['Décalage_Horaire'].value_counts().get('+02:00', 0))
print(df_objets['Décalage_Horaire'].value_counts().get('+01:00', 0))

562419
344857


In [150]:
# Extraire l'année, le mois, le jour, l'heure et le décalage horaire de 'Date et heure de restitution'
df_objets['RAnnée'] = df_objets['Date et heure de restitution'].str[:4]            # Les 4 premiers caractères pour l'année
df_objets['RMois'] = df_objets['Date et heure de restitution'].str[5:7]            # Les caractères 5 à 6 pour le mois
df_objets['RJour'] = df_objets['Date et heure de restitution'].str[8:10]           # Les caractères 8 à 9 pour le jour
df_objets['RHeure'] = df_objets['Date et heure de restitution'].str[11:19]         # Les caractères 11 à 18 pour l'heure:minute:seconde
df_objets['RDécalage_Horaire'] = df_objets['Date et heure de restitution'].str[-6:]  # Les 6 derniers caractères pour le décalage horaire

# Affichage des premières lignes pour vérifier les nouvelles colonnes
print(df_objets.head())


                        Date Date et heure de restitution                Gare  \
0  2018-08-16T08:46:55+02:00    2018-08-16T10:52:08+02:00              Nantes   
1  2018-08-16T08:50:34+02:00                          NaN               Redon   
2  2018-08-16T09:11:18+02:00                          NaN             Béziers   
3  2018-08-16T09:19:47+02:00                          NaN              Rennes   
4  2018-08-16T09:23:18+02:00                          NaN  Paris Montparnasse   

                                Nature d'objets  \
0                            Téléphone portable   
1                           Casque (vélo, moto)   
2         Sac d'enseigne (plastique, papier, …)   
3  Manteau, veste, blazer, parka, blouson, cape   
4                                     Sac à dos   

                                       Type d'objets Type d'enregistrement  \
0  Appareils électroniques, informatiques, appare...        Objet restitué   
1           Vélos, trottinettes, accessoires 2 rou

In [152]:
# Convertir les colonnes 'Date' et 'Date et heure de restitution' au format datetime avec UTC
df_objets['Date'] = pd.to_datetime(df_objets['Date'], utc=True)
df_objets['Date et heure de restitution'] = pd.to_datetime(df_objets['Date et heure de restitution'], utc=True)

# Filtrer les lignes où 'Date et heure de restitution' n'est pas NaT
valid_dates = df_objets['Date et heure de restitution'].notna()

# Vérifier que 'Date et heure de restitution' est après 'Date' uniquement pour les lignes valides
date_check = df_objets.loc[valid_dates, 'Date et heure de restitution'] > df_objets.loc[valid_dates, 'Date']

# Afficher le résultat
if date_check.all():
    print("Toutes les valeurs valides de 'Date et heure de restitution' sont après 'Date'.")
else:
    print("Certaines valeurs valides de 'Date et heure de restitution' ne sont pas après 'Date'.")
    print(df_objets.loc[valid_dates & ~date_check, ['Date', 'Date et heure de restitution']])  # Afficher les lignes concernées


Toutes les valeurs valides de 'Date et heure de restitution' sont après 'Date'.


In [156]:
# Trier les couples (jour, mois) pour chaque décalage horaire par mois, puis par jour
couples_plus_01_sorted = (
    df_objets[df_objets['Décalage_Horaire'] == '+01:00'][['Jour', 'Mois']]
    .drop_duplicates()
    .sort_values(by=['Mois', 'Jour'])
    .values.tolist()
)

couples_plus_02_sorted = (
    df_objets[df_objets['Décalage_Horaire'] == '+02:00'][['Jour', 'Mois']]
    .drop_duplicates()
    .sort_values(by=['Mois', 'Jour'])
    .values.tolist()
)

print("Couples (jour, mois) avec décalage horaire +01:00 triés :", couples_plus_01_sorted)
print("Couples (jour, mois) avec décalage horaire +02:00 triés :", couples_plus_02_sorted)


Couples (jour, mois) avec décalage horaire +01:00 triés : [['01', '01'], ['02', '01'], ['03', '01'], ['04', '01'], ['05', '01'], ['06', '01'], ['07', '01'], ['08', '01'], ['09', '01'], ['10', '01'], ['11', '01'], ['12', '01'], ['13', '01'], ['14', '01'], ['15', '01'], ['16', '01'], ['17', '01'], ['18', '01'], ['19', '01'], ['20', '01'], ['21', '01'], ['22', '01'], ['23', '01'], ['24', '01'], ['25', '01'], ['26', '01'], ['27', '01'], ['28', '01'], ['29', '01'], ['30', '01'], ['31', '01'], ['01', '02'], ['02', '02'], ['03', '02'], ['04', '02'], ['05', '02'], ['06', '02'], ['07', '02'], ['08', '02'], ['09', '02'], ['10', '02'], ['11', '02'], ['12', '02'], ['13', '02'], ['14', '02'], ['15', '02'], ['16', '02'], ['17', '02'], ['18', '02'], ['19', '02'], ['20', '02'], ['21', '02'], ['22', '02'], ['23', '02'], ['24', '02'], ['25', '02'], ['26', '02'], ['27', '02'], ['28', '02'], ['29', '02'], ['01', '03'], ['02', '03'], ['03', '03'], ['04', '03'], ['05', '03'], ['06', '03'], ['07', '03'], ['0

In [158]:
# Récupérer les colonnes 'Nature d\'objets' et 'Type d\'objets'
df_nature_type = df_objets[['Nature d\'objets', 'Type d\'objets']]

# Supprimer les doublons
unique_pairs = df_nature_type.drop_duplicates()

# Vérifier si chaque 'Nature d\'objets' est associée à un seul 'Type d\'objets'
nature_to_type = unique_pairs.groupby('Nature d\'objets')['Type d\'objets'].nunique()

# Sélectionner les 'Nature d\'objets' ayant plus d'un type d'objets associé
natures_with_multiple_types = nature_to_type[nature_to_type > 1].index

# Filtrer les couples uniques pour les 'Nature d\'objets' ayant plus d'un type d'objets associé
multiple_type_pairs = unique_pairs[unique_pairs['Nature d\'objets'].isin(natures_with_multiple_types)]

# Trier les résultats par "Nature d'objets" dans l'ordre alphabétique
sorted_pairs = multiple_type_pairs.sort_values(by='Nature d\'objets')

# Convertir en liste de tuples
result = list(sorted_pairs.itertuples(index=False, name=None))

# Afficher la liste de tuples
print("Couples uniques de 'Nature d'objets' et 'Type d'objets' (triés par 'Nature d'objets') :")
print(result)

Couples uniques de 'Nature d'objets' et 'Type d'objets' (triés par 'Nature d'objets') :
[('Autres divers', 'Divers'), ('Autres divers', "Articles d'enfants, de puériculture"), ('Autres divers', "Pièces d'identités et papiers personnels"), ('Autres vêtements', 'Vêtements, chaussures'), ('Autres vêtements', 'Optique'), ("Carte d'abonnement", "Pièces d'identités et papiers personnels"), ("Carte d'abonnement", 'Porte-monnaie / portefeuille, argent, titres'), ('Chéquier', 'Porte-monnaie / portefeuille, argent, titres'), ('Chéquier', 'Bagagerie: sacs, valises, cartables'), ('Manteau, veste, blazer, parka, blouson, cape', 'Vêtements, chaussures'), ('Manteau, veste, blazer, parka, blouson, cape', 'Appareils électroniques, informatiques, appareils photo'), ('Porte-monnaie, portefeuille', 'Porte-monnaie / portefeuille, argent, titres'), ('Porte-monnaie, portefeuille', "Pièces d'identités et papiers personnels")]


Il y existe des natures d'objets qui ont plusieurs types de d'objets associé. Type d'objets peut-il est considéré comme étant une dimension en plus ? 

In [70]:
df_missing_gare = df_objets[df_objets['Gare'].isna()]
print(df_missing_gare.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1079 entries, 11561 to 907275
Data columns (total 7 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Date                          1079 non-null   object 
 1   Date et heure de restitution  186 non-null    object 
 2   Gare                          0 non-null      object 
 3   Code UIC                      0 non-null      float64
 4   Nature d'objets               1079 non-null   object 
 5   Type d'objets                 1079 non-null   object 
 6   Type d'enregistrement         1079 non-null   object 
dtypes: float64(1), object(6)
memory usage: 67.4+ KB
None


In [167]:
# Remplacer les valeurs NaN dans la colonne 'Gare' par 'Gare Inconnue'
df_objets['Gare'] = df_objets['Gare'].fillna("Gare Inconnue")

## Changement de sujet

In [169]:
sample_df = df_objets.sample(n=100000, random_state=42)

# Afficher un aperçu de l'échantillon
print(sample_df.head())
print(sample_df.info())

                            Date Date et heure de restitution  \
769954 2015-06-15 09:35:44+00:00                          NaT   
539855 2023-11-13 09:07:26+00:00    2023-12-05 14:53:38+00:00   
14585  2019-08-21 13:48:14+00:00    2019-08-21 14:17:49+00:00   
165670 2022-12-16 13:23:05+00:00                          NaT   
293243 2022-01-26 19:49:35+00:00    2022-03-30 08:47:16+00:00   

                      Gare                        Nature d'objets  \
769954  Paris Saint-Lazare                       Pull over, gilet   
539855       Lyon Perrache                              Sac à dos   
14585            Paris Est  Sac d'enseigne (plastique, papier, …)   
165670  Paris Montparnasse        Autre pièce ou papier personnel   
293243  Paris Gare de Lyon              Valise, sac sur roulettes   

                                   Type d'objets Type d'enregistrement Année  \
769954                     Vêtements, chaussures          Objet trouvé  2015   
539855       Bagagerie: sacs, vali

In [173]:
# Filtrer les observations sans NaN dans 'Décalage_Horaire' et 'RDécalage_Horaire' puis comparer
different_timezone_pairs = df_objets[
    df_objets['Décalage_Horaire'].notna() &
    df_objets['RDécalage_Horaire'].notna() &
    (df_objets['Décalage_Horaire'] != df_objets['RDécalage_Horaire'])
]

# Compter et afficher les résultats
count_different_timezone_pairs = different_timezone_pairs.shape[0]
different_timezone_pairs_head = different_timezone_pairs[['Décalage_Horaire', 'RDécalage_Horaire']].head()

print("Nombre d'observations avec des couples Décalage_Horaire et RDécalage_Horaire différents (sans NaN) :", count_different_timezone_pairs)
print("Exemple des premières différences trouvées :")
print(different_timezone_pairs_head)


Nombre d'observations avec des couples Décalage_Horaire et RDécalage_Horaire différents (sans NaN) : 19467
Exemple des premières différences trouvées :
    Décalage_Horaire RDécalage_Horaire
209           +02:00            +01:00
252           +02:00            +01:00
394           +02:00            +01:00
592           +02:00            +01:00
681           +02:00            +01:00
